In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

import numbers
from sklearn import model_selection


In [6]:

data_for_use = pd.read_csv('./ethereum_data_for_use.csv', index_col=0)
data_btc = pd.read_csv('./bitcoin_data_for_use_v2.csv', index_col=0)
data_gold = pd.read_csv('./gold_data_for_use.csv', index_col=0)
data_nasdaq = pd.read_csv('./nasdaq_data_for_use.csv', index_col=0)

#los datos de volumen y el dia del bitcoin no nos interesan para cruzar con ETH
data_btc = data_btc.drop(labels=['dia', 'subida',
                    'varV0','varV1','varV2','varV3','varV4','varV5','varV6','varV7','varV8','varV9',
                    'varV10','varV11','varV12','varV13','varV14','varV15','varV16','varV17','varV18','varV19',
                    'varV20','varV21','varV22','varV23','varV24','varV25','varV26','varV27','varV28','varV29'
                    ], axis=1)

data_for_use_basic = data_for_use.drop(labels=['dia',
                    'varV0','varV1','varV2','varV3','varV4','varV5','varV6','varV7','varV8','varV9',
                    'varV10','varV11','varV12','varV13','varV14','varV15','varV16','varV17','varV18','varV19',
                    'varV20','varV21','varV22','varV23','varV24','varV25','varV26','varV27','varV28','varV29'
                    ], axis=1)


data_for_use_w_gold = data_for_use.merge(data_gold, left_index=True, right_index=True)
data_for_use_w_nasdaq = data_for_use.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_w_btc = data_for_use_basic.merge(data_btc, left_index=True, right_index=True)

data_for_use_w_all = data_for_use_w_gold.merge(data_nasdaq, left_index=True, right_index=True)



data_for_use_bone_deep = data_for_use_basic.drop(labels=['varPm93',
                    'varPm123','varPm153','varPm183','varPm213','varPm243','varPm273','varPm303','varPm333',
                    'varPs30','varPs37','varPs44','varPs51','varPs58','varPs65','varPs72','varPs79','varPs86'
                    ], axis=1)


In [7]:
data_for_use_w_btc

,varP0_x,varP1_x,varP2_x,varP3_x,varP4_x,varP5_x,varP6_x,varP7_x,varP8_x,varP9_x,...,varPs86_y,varPm93_y,varPm123_y,varPm153_y,varPm183_y,varPm213_y,varPm243_y,varPm273_y,varPm303_y,varPm333_y
2017-01-01,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,0.013799,-0.012482,0.013901,-0.055221,...,0.012113,0.059539,-0.050802,-0.138454,0.308064,0.194686,0.069850,0.024249,0.110799,-0.145794
2017-01-02,0.025199,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,0.013799,-0.012482,0.013901,...,0.008347,0.072829,0.045369,-0.168825,0.157187,0.274157,0.059979,0.052111,0.028173,-0.098079
2017-01-03,0.160737,0.025199,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,0.013799,-0.012482,...,0.009593,0.061430,0.016212,-0.171586,0.193696,0.278477,0.056472,0.040036,0.054736,-0.099174
2017-01-04,0.156947,0.160737,0.025199,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,0.013799,...,0.011208,0.023271,0.034452,-0.137689,0.166355,0.251030,0.085472,0.021944,0.100390,-0.177986
2017-01-05,-0.088627,0.156947,0.160737,0.025199,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,...,0.050586,0.002580,0.058415,-0.151016,0.156769,0.276971,0.084663,0.021192,0.099177,-0.169031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-21,-0.065753,-0.111266,-0.029886,0.009851,-0.048374,-0.012167,0.054273,0.043725,-0.036545,0.042766,...,-0.029375,-0.043061,-0.307969,-0.011426,-0.027993,0.819894,0.296122,0.294667,0.420558,0.264591
2021-09-22,0.113382,-0.065753,-0.111266,-0.029886,0.009851,-0.048374,-0.012167,0.054273,0.043725,-0.036545,...,0.087056,-0.156113,-0.274826,-0.054379,-0.048679,0.743330,0.420131,0.265584,0.400979,0.219860
2021-09-23,0.025230,0.113382,-0.065753,-0.111266,-0.029886,0.009851,-0.048374,-0.012167,0.054273,0.043725,...,0.103432,-0.065139,-0.319474,-0.031845,-0.026437,0.690406,0.351016,0.242234,0.466289,0.217606
2021-09-24,-0.070941,0.025230,0.113382,-0.065753,-0.111266,-0.029886,0.009851,-0.048374,-0.012167,0.054273,...,0.039077,-0.128739,-0.226667,-0.031976,0.058981,0.512089,0.309128,0.316711,0.432640,0.215800


In [8]:
#aqui se marca lo que se va a usar de verdad para el resto del programa
data_for_use = data_for_use_w_btc

In [9]:
# Partimos los datos en train y test por fechas, sin que sea aleatorio, que es como funcionaría
# en la realidad el modelo
data_no_test, data_test = train_test_split(data_for_use, shuffle=False, test_size=0.2)

# Partimos los datos de train nuevamente en validation y train
data_train, data_valid = train_test_split(data_no_test, shuffle=False, test_size=0.2)

In [10]:
# Separamos la variable que queremos predecir
X_train=data_train.drop(labels=['subida'], axis=1)
y_train=data_train['subida']

X_valid=data_valid.drop(labels=['subida'], axis=1)
y_valid=data_valid['subida']

X_test=data_test.drop(labels=['subida'], axis=1)
y_test=data_test['subida']

In [11]:
# Miramos cuántos días sube el eth sobre el total en cada uno de los segmentos
# esto nos servirá para hacernos una idea de cómo de buenos son los resultados de los modelos
y_train.sum()/y_train.size

0.49909584086799275

In [12]:
y_valid.sum()/y_valid.size

0.5703971119133574

In [13]:
y_test.sum()/y_test.size

0.5664739884393064

In [14]:
#Tenemos que hacer one-hot encoding sobre la columna "dia"
#Sobre el resto aplicamos el standardscaler

number_columns = data_for_use.select_dtypes('number').columns

t=[
    #('dia', 
    #OneHotEncoder(handle_unknown='ignore'),
    #['dia']),
    ('scaler', StandardScaler(),number_columns)
    ]

ct = ColumnTransformer(transformers=t, remainder='passthrough')

#data_for_use_t=ct.fit_transform(data_for_use)

#X_train=ct.fit_transform(X_train)
#X_valid=ct.transform(X_valid)
#X_test=ct.transform(X_test)

In [15]:
def walk_forward_validation (model, X_train_wfv, y_train_wfv, X_test_wfv, y_test_wfv):
    y_pred_wfv = list()
    for i in range(len(y_test_wfv)):
        X_train_wfv_ct=ct.fit_transform(X_train_wfv)
        model.fit(X_train_wfv_ct, y_train_wfv)
        X_test_wfv_ct=ct.transform(X_test_wfv)
        y_pred_next = model.predict(X_test_wfv_ct[i:i+1])
        y_pred_wfv.append(y_pred_next[0])
        X_train_wfv=X_train_wfv.append(X_test_wfv[i:i+1])
        y_train_wfv=y_train_wfv.append(pd.Series(y_test_wfv[i]))
    
    return metrics.confusion_matrix(y_test_wfv, y_pred_wfv), metrics.f1_score(y_test_wfv,y_pred_wfv), metrics.accuracy_score(y_test_wfv,y_pred_wfv)
        
        

In [16]:
lr = LogisticRegression(penalty='l2', solver='lbfgs')

X_train_c=X_train.copy()
y_train_c=y_train.copy()
X_valid_c=X_valid.copy()
y_valid_c=y_valid.copy()


conf_mat, f1sc, accsc = walk_forward_validation(lr, X_train_c, y_train_c, X_valid_c, y_valid_c)

In [17]:
print(conf_mat, f1sc, accsc)

[[59 60]
 [79 79]] 0.531986531986532 0.4981949458483754


In [18]:
for vecinos in range(1,31):
    kn = KNeighborsClassifier(n_neighbors=vecinos)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(kn, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Numero de vecinos aplicado: " + str(vecinos))
    print(conf_mat, f1sc, accsc)

Numero de vecinos aplicado: 1
[[45 74]
 [79 79]] 0.5080385852090032 0.44765342960288806
Numero de vecinos aplicado: 2
[[ 78  41]
 [106  52]] 0.4143426294820717 0.4693140794223827
Numero de vecinos aplicado: 3
[[47 72]
 [71 87]] 0.5488958990536278 0.48375451263537905
Numero de vecinos aplicado: 4
[[ 69  50]
 [102  56]] 0.42424242424242425 0.45126353790613716
Numero de vecinos aplicado: 5
[[56 63]
 [69 89]] 0.5741935483870967 0.5234657039711191
Numero de vecinos aplicado: 6
[[75 44]
 [95 63]] 0.47547169811320755 0.4981949458483754
Numero de vecinos aplicado: 7
[[48 71]
 [63 95]] 0.5864197530864197 0.516245487364621
Numero de vecinos aplicado: 8
[[68 51]
 [80 78]] 0.5435540069686412 0.5270758122743683
Numero de vecinos aplicado: 9
[[48 71]
 [60 98]] 0.5993883792048929 0.5270758122743683
Numero de vecinos aplicado: 10
[[62 57]
 [82 76]] 0.5223367697594502 0.4981949458483754
Numero de vecinos aplicado: 11
[[47 72]
 [63 95]] 0.5846153846153846 0.5126353790613718
Numero de vecinos aplicado: 1

In [19]:
for grado in range(1,4):
    sv = SVC(probability=True, kernel='poly', degree=grado)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(sv, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Grado aplicado: " + str(grado))
    print(conf_mat, f1sc, accsc)

Grado aplicado: 1
[[64 55]
 [73 85]] 0.5704697986577181 0.5379061371841155
Grado aplicado: 2
[[37 82]
 [59 99]] 0.584070796460177 0.49097472924187724
Grado aplicado: 3
[[44 75]
 [60 98]] 0.5921450151057402 0.5126353790613718


In [20]:
for profund in range(1,31):
    dt = DecisionTreeClassifier(max_depth=profund)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(dt, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Maxima profundidad aplicada: " + str(profund))
    print(conf_mat, f1sc, accsc)

Maxima profundidad aplicada: 1
[[ 98  21]
 [112  46]] 0.4088888888888889 0.51985559566787
Maxima profundidad aplicada: 2
[[ 83  36]
 [100  58]] 0.4603174603174603 0.5090252707581228
Maxima profundidad aplicada: 3
[[ 99  20]
 [125  33]] 0.31279620853080564 0.47653429602888087
Maxima profundidad aplicada: 4
[[ 93  26]
 [116  42]] 0.3716814159292035 0.48736462093862815
Maxima profundidad aplicada: 5
[[ 90  29]
 [111  47]] 0.40170940170940167 0.49458483754512633
Maxima profundidad aplicada: 6
[[71 48]
 [87 71]] 0.5126353790613719 0.5126353790613718
Maxima profundidad aplicada: 7
[[80 39]
 [84 74]] 0.5461254612546126 0.555956678700361
Maxima profundidad aplicada: 8
[[65 54]
 [81 77]] 0.5328719723183392 0.5126353790613718
Maxima profundidad aplicada: 9
[[71 48]
 [81 77]] 0.5441696113074205 0.5342960288808665
Maxima profundidad aplicada: 10
[[63 56]
 [75 83]] 0.5589225589225589 0.5270758122743683
Maxima profundidad aplicada: 11
[[65 54]
 [79 79]] 0.5429553264604811 0.51985559566787
Maxima pro

In [21]:
for estimadores in range(1,21):
    adab = AdaBoostClassifier(n_estimators=estimadores)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(adab, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Estimadores aplicados: " + str(estimadores))
    print(conf_mat, f1sc, accsc)

Estimadores aplicados: 1
[[ 98  21]
 [112  46]] 0.4088888888888889 0.51985559566787
Estimadores aplicados: 2
[[ 97  22]
 [111  47]] 0.41409691629955947 0.51985559566787
Estimadores aplicados: 3
[[ 93  26]
 [107  51]] 0.4340425531914893 0.51985559566787
Estimadores aplicados: 4
[[ 92  27]
 [107  51]] 0.43220338983050843 0.516245487364621
Estimadores aplicados: 5
[[59 60]
 [75 83]] 0.5514950166112956 0.5126353790613718
Estimadores aplicados: 6
[[ 81  38]
 [105  53]] 0.42570281124497994 0.48375451263537905
Estimadores aplicados: 7
[[60 59]
 [80 78]] 0.5288135593220339 0.4981949458483754
Estimadores aplicados: 8
[[62 57]
 [81 77]] 0.5273972602739727 0.5018050541516246
Estimadores aplicados: 9
[[64 55]
 [76 82]] 0.5559322033898305 0.5270758122743683
Estimadores aplicados: 10
[[60 59]
 [83 75]] 0.5136986301369862 0.48736462093862815
Estimadores aplicados: 11
[[67 52]
 [81 77]] 0.5365853658536586 0.51985559566787
Estimadores aplicados: 12
[[64 55]
 [72 86]] 0.5752508361204013 0.54151624548736

In [22]:
dt = DecisionTreeClassifier(max_depth=7)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
dt.fit(X_train_c,y_train_c)
y_predict=dt.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[580,  93],
        [253, 457]]),
 0.7253968253968255,
 0.749819233550253)

In [23]:
sv = SVC(probability=True, kernel='poly', degree=1)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
sv.fit(X_train_c,y_train_c)
y_predict=sv.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[380, 293],
        [232, 478]]),
 0.6455097906819716,
 0.6203904555314533)

In [24]:
adab = AdaBoostClassifier(n_estimators=13)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
adab.fit(X_train_c,y_train_c)
y_predict=adab.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[450, 223],
        [295, 415]]),
 0.615727002967359,
 0.6254519161243673)

In [25]:
kn = KNeighborsClassifier(n_neighbors=17)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
kn.fit(X_train_c,y_train_c)
y_predict=kn.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[320, 353],
        [220, 490]]),
 0.6310367031551835,
 0.5856832971800434)

In [26]:
kn = KNeighborsClassifier(n_neighbors=23)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
kn.fit(X_train_c,y_train_c)
y_predict=kn.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[300, 373],
        [219, 491]]),
 0.6238881829733165,
 0.571945046999277)

In [27]:
#Este es el código para ver cómo funciona el modelo de verdad

adab = AdaBoostClassifier(n_estimators=13)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)
X_test_c=X_test.copy()
y_test_c=y_test.copy()

conf_mat, f1sc, accsc = walk_forward_validation(adab, X_train_c, y_train_c, X_test_c, y_test_c)

print(conf_mat, f1sc, accsc)

[[ 75  75]
 [ 87 109]] 0.5736842105263158 0.5317919075144508
